# ALBEF 的模型结构

## 图像编码器（Image Encoder）
采用 12 层的 ViT-B/16（Vision Transformer base 模型， patch 大小为 16×16）作为图像编码器。其作用是将输入图像转换为视觉特征嵌入（embedding），并通过自注意力机制（Self-Attention）捕捉图像的全局信息，最终输出包含图像全局信息的 CLS（Classification Token）嵌入。
## 文本编码器（Text Encoder）
基于 BERT 模型的前 6 层构成。其功能是对输入文本进行编码，生成文本特征嵌入，同样通过自注意力机制捕捉文本的上下文信息，并输出包含文本全局信息的 CLS 嵌入。
## 多模态编码器（Multimodal Encoder）
采用 BERT 模型的后 6 层作为多模态融合模块。图像编码器输出的视觉特征嵌入和文本编码器输出的文本特征嵌入会被输入到该模块，通过交叉注意力机制（Cross-Attention）实现视觉与文本信息的深度融合，为后续的多模态任务（如图文匹配、掩码语言建模）提供融合特征。


# ALBEF 的创新点
1. 摒弃目标检测器（Object Detector），提升效率与泛化性
不同于之前基于检测器的视觉 - 语言预训练方法（如 Oscar），ALBEF 直接使用 ViT 作为图像编码器，避免了检测器带来的高计算复杂度（大量 anchor 和 bounding box 计算），同时摆脱了检测器性能的限制（检测器依赖有限类别标注数据集，导致模型泛化性受限），显著提升了训练效率和模型的泛化能力。
2. 引入动量模型（Momentum Model）与动量蒸馏（Momentum Distillation）动量模型通过指数移动平均（EMA），作为 “教师模型” 提供平滑、稳定的软目标（soft target）。动量蒸馏通过 KL 散度约束 “学生模型”（主模型）的预测分布与教师模型对齐，弥补了原始数据中图文对噪声（如匹配度低的样本）的影响，提升了训练稳定性和泛化能力。
3. 硬负样本（Hard Negatives）的应用
在 ITM 任务中，负样本来自图文对比学习中相似度次高的样本（基于 InfoNCE 损失筛选），这类样本学习难度更大，能有效增强模型对细微差异的辨别能力，提升图文匹配任务的性能。
4. 高效的多模态融合架构
将 BERT 模型拆分为前 6 层（文本编码）和后 6 层（多模态融合），使文本编码与多模态融合共享参数，既保证了文本语义的充分提取，又通过交叉注意力实现了图文信息的高效交互，简化了模型结构同时提升了融合效果。
